In [5]:
#basic package
import tqdm
import os
import numpy as np
import glob
import random
import pandas as pd
import sys
import time
import datetime as dt
import itertools
import re
import math
from collections import Counter
import operator
from scipy.stats import entropy
import networkit as nk #more efficient
import pickle
import statistics
from sklearn.utils import shuffle
from scipy.stats import pearsonr, spearmanr

#plot
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits import mplot3d

In [6]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import preprocessing_Origins,is_day,time_series_henColumn_tsRow, is_WG_open, is_day
import config_exp2 as config

In [7]:
print('change the configuration file if not done yet!')
path_extracted_data = config.path_extracted_data
path_initial_data = config.path_initial_data
id_run = config.id_run
dico_matching = config.dico_matching
li_tracking_date = config.li_tracking_date
print(id_run)
path_extracted_data_visual = os.path.join(path_extracted_data,'LongformatModels')
#create a director if not existing
if not os.path.exists(path_extracted_data_visual):
    os.makedirs(path_extracted_data_visual)

change the configuration file if not done yet!
EXP2_


# Downlaod data

In [14]:
path_extracted_data

'G:\\VPHI\\Welfare\\2- Research Projects\\Camille Montalcini\\Origins.GS\\DataOutput\\TrackingSystem\\EXP2_'

In [27]:
#new data
df = pd.read_csv(os.path.join(path_extracted_data, 'combidecjan.csv'), sep=',',parse_dates=['date'], dayfirst=True) 
df[['Serial_Sensor_1_Ammonia litter wintergarden','Serial_Sensor_3_Ammonia litter corridor']]
df.rename(columns={'Serial_Sensor_1_Ammonia litter wintergarden':'AmmoniaWGlitter',
                  'Serial_Sensor_3_Ammonia litter corridor':'Ammonialitter','litter':'totminHens'}, inplace=True)
df = df[['Hour','date','AmmoniaWGlitter','Ammonialitter','totminHens']].copy()
print(df.shape)
df.head(3)

(768, 5)


,Hour,date,AmmoniaWGlitter,Ammonialitter,totminHens
0,0,2022-12-19,24,31.0,NaN
1,1,2022-12-19,23,21.0,0.0
2,2,2022-12-19,25,25.0,977.0


In [49]:
df['Timestamp'] = df.apply(lambda x: dt.datetime(x['date'].year,x['date'].month,x['date'].day,x['Hour'],0,0), axis=1)
df['time'] = df['Timestamp'].map(lambda x: (x-df['Timestamp'].min()).total_seconds()/60/60)

In [52]:
df['id'] =1

In [53]:
df

,Hour,date,AmmoniaWGlitter,Ammonialitter,totminHens,time,Timestamp,id
0,0,2022-12-19,24,31.0,NaN,0.0,2022-12-19 00:00:00,1
1,1,2022-12-19,23,21.0,0.0,1.0,2022-12-19 01:00:00,1
2,2,2022-12-19,25,25.0,977.0,2.0,2022-12-19 02:00:00,1
3,3,2022-12-19,28,31.0,532646.0,3.0,2022-12-19 03:00:00,1
4,4,2022-12-19,23,15.0,946670.0,4.0,2022-12-19 04:00:00,1
...,...,...,...,...,...,...,...,...
763,19,2023-01-19,23,29.0,0.0,763.0,2023-01-19 19:00:00,1
764,20,2023-01-19,21,19.0,0.0,764.0,2023-01-19 20:00:00,1
765,21,2023-01-19,26,25.0,0.0,765.0,2023-01-19 21:00:00,1
766,22,2023-01-19,20,25.0,0.0,766.0,2023-01-19 22:00:00,1


In [54]:
df.to_csv(os.path.join(path_extracted_data,'ENVFBL.csv'), index=False, sep=';')

In [ ]:
# new

In [56]:
df.head(24)

,Hour,date,AmmoniaWGlitter,Ammonialitter,totminHens,time,Timestamp,id
0,0,2022-12-19,24,31.0,NaN,0.0,2022-12-19 00:00:00,1
1,1,2022-12-19,23,21.0,0.0,1.0,2022-12-19 01:00:00,1
2,2,2022-12-19,25,25.0,977.0,2.0,2022-12-19 02:00:00,1
3,3,2022-12-19,28,31.0,532646.0,3.0,2022-12-19 03:00:00,1
4,4,2022-12-19,23,15.0,946670.0,4.0,2022-12-19 04:00:00,1
5,5,2022-12-19,20,13.0,714135.0,5.0,2022-12-19 05:00:00,1
6,6,2022-12-19,19,13.0,621908.0,6.0,2022-12-19 06:00:00,1
7,7,2022-12-19,19,12.0,691019.0,7.0,2022-12-19 07:00:00,1
8,8,2022-12-19,18,14.0,818610.0,8.0,2022-12-19 08:00:00,1
9,9,2022-12-19,20,15.0,1018191.0,9.0,2022-12-19 09:00:00,1


In [58]:
print(df.shape)
df_day = df[(df['Hour']>3)&(df['Hour']<17)].copy()
print(df_day.shape)

(768, 8)
(416, 8)


In [60]:
df_day['Hour'] = df_day['Hour'].map(lambda x: x-4)

In [62]:
df_day

,Hour,date,AmmoniaWGlitter,Ammonialitter,totminHens,time,Timestamp,id
4,0,2022-12-19,23,15.0,946670.0,4.0,2022-12-19 04:00:00,1
5,1,2022-12-19,20,13.0,714135.0,5.0,2022-12-19 05:00:00,1
6,2,2022-12-19,19,13.0,621908.0,6.0,2022-12-19 06:00:00,1
7,3,2022-12-19,19,12.0,691019.0,7.0,2022-12-19 07:00:00,1
8,4,2022-12-19,18,14.0,818610.0,8.0,2022-12-19 08:00:00,1
...,...,...,...,...,...,...,...,...
756,8,2023-01-19,24,21.0,915853.0,756.0,2023-01-19 12:00:00,1
757,9,2023-01-19,26,17.0,1154928.0,757.0,2023-01-19 13:00:00,1
758,10,2023-01-19,24,23.0,1234293.0,758.0,2023-01-19 14:00:00,1
759,11,2023-01-19,25,21.0,1130064.0,759.0,2023-01-19 15:00:00,1


In [61]:
df_day.to_csv(os.path.join(path_extracted_data,'ENVFBL_day.csv'), index=False, sep=';')